In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib import OCR, CameraSource, OCREngine, PlaneDetector

In [ ]:
plane_detector = PlaneDetector(confidence_threshold=0.5)
ocr = OCR(OCREngine.PADDLEOCR)

In [ ]:
from lib.preprocessing import preprocess_test

In [ ]:
camera = CameraSource("data/IMG_3370.MOV")
# video_file = VideoFileSource("data/IMG_3363.MOV", target_fps=5)

In [ ]:
camera.start_stream_notebook(
    plane_detector,
    ocr,
    target_fps=5,
    start_frame=650,
    preprocessing_func=preprocess_test,
)